In [10]:
from trabalho_etapa1 import  Grafo, quantidadeDeVertices, capacidadeDosVeiculos, quantidadeDeVerticesRequeridos


arquivo = "BHW1.dat"

g = Grafo(quantidadeDeVertices(arquivo),capacidadeDosVeiculos(arquivo))

g.adiciona_arco_Normal(arquivo)
g.adiciona_arco_Requerido(arquivo)
g.adiciona_aresta_Normal(arquivo)
g.adiciona_aresta_requerida(arquivo)
g.floyd_warshall()
[dist,pred] = g.floyd_warshall()


print('Quantidade de vértices: ', quantidadeDeVertices(arquivo))
print('Quantidade de arestas: ', g.adiciona_aresta_Normal(arquivo))
print('Quantidade de arcos: ', g.adiciona_arco_Normal(arquivo))
print('Quantidade de vértices requeridos: ', quantidadeDeVerticesRequeridos(arquivo))
print('Quantidade de arestas requeridas: ', g.adiciona_aresta_requerida(arquivo))
print('Quantidade de arcos requeridas: ', g.adiciona_arco_Requerido(arquivo))
print('Densidade: ', g.densidadeGrafo(arquivo))
print('Componentes Conectados: ', g.encontrarComponentes())
print('Grau mínimo dos vértices: ', g.grauMinimo())
print('Grau máximo dos vértices: ', g.grauMaximo())
print('Intermediação: ', g.intermediacao(pred))
print('Caminho médio: ', g.caminho_medio(dist))
print('Diâmetro: ', g.diametroDoGrafo())

print('Matriz de Distancias: ')
print(g.imprimir_matriz(dist))

print('Matriz de predecessores: ')
print(g.imprimir_matriz(pred))


Quantidade de vértices:  12
Quantidade de arestas:  0
Quantidade de arcos:  11
Quantidade de vértices requeridos:  7
Quantidade de arestas requeridas:  11
Quantidade de arcos requeridas:  11
Densidade:  0.3333333333333333
Componentes Conectados:  [[0, 1, 3, 6, 9, 11, 2, 8, 5, 7, 10, 4]]
Grau mínimo dos vértices:  3
Grau máximo dos vértices:  5
Intermediação:  [17, 17, 8, 3, 12, 19, 11, 15, 9, 6, 5, 10]
Caminho médio:  30.37121212121212
Diâmetro:  59
Matriz de Distancias: 
0      27     42     35     31     16     20     37     32     39     58     9      
13     0      37     19     42     29     33     41     5      38     34     22     
27     37     0      41     5      20     24     41     42     48     46     23     
17     19     20     0      25     33     37     54     24     56     53     26     
22     48     11     52     0      15     19     36     53     43     41     18     
7      34     26     42     15     0      4      21     39     28     42     3      
16     43    